# Funciones de utilidad para interacción con data

## Importaciones y funciones requeridas

In [44]:
import csv
import json
import time
import io
import pandas as pd

from datetime import datetime

La función `normalize_str` es usada para limpiar los espacios en blanco al inicio y/o final de una cadena. También convirte algunas secuencias de caracteres innecesarios por guión bajo (`_`), o en otros casos, solo se remueven.

In [45]:
def normalize_str(original_str: str):
    return original_str.strip().lower().replace(' - ', '_').replace(', ', '_').replace(' ', '_').replace('\ufeff', '').replace('(', '').replace(')', '').replace('.', '_')

La función `parse_datetime` se encarga de recibir una fecha en formato `MM/dd/YYYY hh:mm:ss pm/am` y lo transforma en un formato de 24 horas con la siguiente apariencia `YYYY-MM-dd hh:mm:ss`.

In [46]:
def parse_datetime(original_date: str):
    return '' if original_date == '' else datetime.strptime(original_date, '%m/%d/%Y %I:%M:%S %p').strftime('%Y-%m-%d %H:%M:%S')

La función `format_value_to_sql` se encarga de convertir un valor en un formato SQL válido para las sentencias de DML.

In [47]:
def format_value_to_sql(value):
    if value is None or value == '':
        return 'NULL'
    elif isinstance(value, str):
        return f"'{value}'"
    elif isinstance(value, bool):
        return str(int(value))
    elif isinstance(value, (int, float)):
        return str(value)
    else:
        raise TyperError(f"Unsupported value type: {type(value)}")

La función `open_json_file` se encarga de abrir un archivo y retornar su información en formato JSON.

In [48]:
def open_json_file(path):
     with open(path, mode="r", encoding="utf-8") as file:
        return json.load(file)

La función `safe_json_file` se encarga de guardar un archivo JSON con la data y en la ruta especificada en los parámetros.

In [49]:
def safe_json_file(path, data):
    with open(path, mode='w', encoding="utf-8") as file:
        file.write(json.dumps(data, indent=4))

## Extracción de archivos CSV desde un Excel

Mediante las siguiente líneas de código, se lográ dividir un archivo excel en diferentes archivos csv, para luego ser usados en la extracción de los datos.

In [50]:
excel_file = pd.ExcelFile("JPM-V2-20231122161725364_2326466888.xlsx")

sheet_names = excel_file.sheet_names

new_names = {
    "Users": 'company-users', 
    "UserEntitlements": 'users-entitlements', 
    "Client": 'products-accounts',
    "Summary": "summary"
}

for sheet in sheet_names:
    dataframe = excel_file.parse(sheet)
    dataframe.to_csv("csv/" + new_names[sheet] + ".csv", index=False)

## Conversión de Archivos CSV a JSON

La función `csv_to_json` es la responsable de recibir un solo archivo en formato CSV y convertirlo en formato JSON. Se debe definir una ruta de entrada para la data original, y una ruta de salida para la data transformada. Es importante recalcar que, a través de este método se realiza un tratamiento especial a la sintaxís de las llaves.

In [51]:
def csv_to_json(csv_path, json_path):
    with open(csv_path, encoding="utf-8") as csv_file:
        csv_reader = csv.DictReader(csv_file)

        json_arr = [
            {
                normalize_str(key): value
                for key, value in row.items()
            }
            for row in csv_reader
        ]
    
    safe_json_file(path=json_path, data=json_arr)

La estrategia que se usa en la siguiente celda es, crear un arreglo con los nombres genericos de los archivos, y cada nombre es usado tanto para los archivos CSV cómo para los JSON. Para evitar confusiones en el directorio de archivos, se guarda cada archivo en una carpeta para su respectivo grupo de formato.

In [52]:
io_files_names = ['company-users', 'users-entitlements', 'products-accounts']

start = time.perf_counter()

for i in range(0, len(io_files_names)):
    csv_to_json(
        csv_path = f"csv/{io_files_names[i]}.csv", 
        json_path=f"json/{io_files_names[i]}.json"
    )

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0594 seconds


## Extracción de Accounts

La función `extract_accounts` está encargada de recibir un arreglo de rutas para cargar de manera conjunta la información de todos los archivos recibidos, y luego crear un arreglo de objetos con las propiedades requeridas en el archivo JSON de salida.

In [53]:
def extract_accounts(source_data_paths, output_data_path):
    data = [item for path in source_data_paths for item in open_json_file(path)]
    accounts = []
    
    for i in data:
        account = {
            'account_name': i['account_name'],
            'account_number': i['account_number'] or '0',
            'account_type': i['account_type'],
            'bank_currency': i['bank_currency'] or None
        }
        # account['id'] = normalize_str(account['account_number'] + '_' + account['account_name'])
        if account not in accounts and account['account_name'] != '':
            accounts.append(account)
    
    safe_json_file(path=output_data_path, data=accounts)

In [54]:
start = time.perf_counter()

extract_accounts(
    source_data_paths = [r"json/users-entitlements.json", r"json/products-accounts.json"],
    output_data_path = r"normalize-json/accounts.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0257 seconds


## Extracción de Products

La función `extract_products` está encargada de recibir un arreglo de rutas para cargar de manera conjunta la información de todos los archivos recibidos, y luego crear un arreglo de objetos con las propiedades requeridas en el archivo JSON de salida.

In [55]:
def extract_products(source_data_paths, output_data_path):
    data = [item for path in source_data_paths for item in open_json_file(path)]
    products = []
    
    for i in data:
        product_name = i['product'].strip().split(', ')
        product = {
            'id': normalize_str(i['product']),
            'product_name': product_name[0],
            'sub_product': product_name[1] if len(product_name) > 1 else None
        }
        if product not in products:
            products.append(product)
            
    safe_json_file(path=output_data_path, data=products)

In [56]:
start = time.perf_counter()

extract_products(
    source_data_paths = [r"json/users-entitlements.json", r"json/products-accounts.json"],
    output_data_path = r"normalize-json/products.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0075 seconds


## Extracción de Functions

La función `extract_functions` está encargada de recibir una ruta para cargar la información del archivo, y luego crear un arreglo de objetos con las propiedades requeridas en el archivo JSON de salida.

In [57]:
def extract_functions(source_data_path, output_data_path):
    data = open_json_file(source_data_path)
    functions = []
    
    for i in data:
        function = {
            'id': normalize_str(i['function_name']),
            'function_name': i['function_name'].strip()
        }
        if function not in functions and function['function_name'] != "":
            functions.append(function)
    
    safe_json_file(path=output_data_path, data=functions)

In [58]:
start = time.perf_counter()

extract_functions(
    source_data_path = r"json/users-entitlements.json",
    output_data_path = r"normalize-json/functions.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0062 seconds


## Normalización de Company Users

La función `normalize_company_users` recibe un archivo json y limpia los valores de la data de origen adaptandose a la tabla del modelo ER de la base de datos. En la data se debe normalizar el valor de la fecha, con el objetivo de hacerla compatible con el valor admitido por SQL.

In [59]:
def normalize_company_users(original_data_path, result_data_path):
    data = open_json_file(original_data_path)
    
    for i in data:
        i['access_id'] = i['accessid']
        i['user_name'] = i['user_name'].strip()
        i['user_status'] = i['user_status'] == 'Active'
        i['user_last_logon_dt'] = parse_datetime(i['user_last_logon_dt'])
            
        del i['accessid']
    
    safe_json_file(path=result_data_path, data=data)

In [60]:
start = time.perf_counter()

normalize_company_users(
    original_data_path = r"json/company-users.json",
    result_data_path = r"normalize-json/company-users.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0128 seconds


## Normalización de ProductsAccounts

La función `normalize_clients` recibe un archivo json y limpia los valores de la data de origen adaptandose a la tabla del modelo ER de la base de datos. Se eliminan las columnas que no son requeridas en la base de datos, y adapta el valor de la columna `product` para la crear la referencia fóranea a la tabla de productos.

In [61]:
def normalize_clients(original_data_path, result_data_path):
    data = open_json_file(original_data_path)

    for i in data:
        i['id'] = normalize_str(i['product'] + '_' + i['account_number'])
        i['product_id'] = normalize_str(i['product'])
        
        del i['product']
        del i['account_name']
        del i['account_type']
        del i['bank_currency']
    
    safe_json_file(path=result_data_path, data=data)

In [62]:
start = time.perf_counter()

normalize_clients(
    original_data_path = r"json/products-accounts.json",
    result_data_path = r"normalize-json/products-accounts.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0025 seconds


## Normalización de Users Entitlements

La función `normalize_users_entitlements` recibe un archivo json y limpia los valores de la data de origen adaptandose a la tabla del modelo ER de la base de datos. Se eliminan las columnas que no son requeridas en la base de datos, y adapta el valor de la columna `product` para la crear la referencia fóranea a la tabla de productos al igual que pasa con `function`.

In [63]:
def normalize_users_entitlements(original_data_path, result_data_path):
    data = open_json_file(original_data_path)

    for i in data:
        i['access_id'] = i['accessid']
        i['product_id'] = normalize_str(i['product'])
        i['function_id'] = normalize_str(i['function_name'])
        i['id'] = normalize_str(i['access_id'] + '_' + i['account_number'] + '_' + i['product_id'] + '_' + i['function_id'] + '_' + i['function_type'])
        
        del i['accessid']
        del i['product']
        del i['function_name']
        del i['account_name']
        del i['account_type']
        del i['bank_currency']

    safe_json_file(path=result_data_path, data=data)

In [64]:
start = time.perf_counter()

normalize_users_entitlements(
    original_data_path = r"json/users-entitlements.json",
    result_data_path = r"normalize-json/users-entitlements.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0184 seconds


## Definición de Perfil - Usuario

![Profiles](./Profiles.png)

La función `get_company_user_index` se encarga de buscar un usuario mediante su `access_id` en un arreglo de objetos y luego retorna el indice en que se encuentra dentro de la lista.

In [65]:
def get_company_user_index(data, access_id):
    for i, d in enumerate(data):
        if d['access_id'] == access_id:
            return i

La función `get_users_functions_products` se encarga de agrupar por usuario las combinaciones de los campos `function_id`, `function_type` y `product_id` de la data normalizada de `user_entitlements`.

In [66]:
def get_users_functions_products(user_entitlements):
    return {
        i['access_id']: [
            {
                'function_id': normalize_str(j['function_id']),
                'function_type': j['function_type'].strip(),
                'product_id': j['product_id'].strip()
            }
            for j in user_entitlements
            if j['access_id'] == i['access_id']
        ]
        for i in user_entitlements
    }

La función `get_unique_user_functions_products` se encarga de obtener los valores únicos por usuario para los campos de `function_id` y `product_id`, por lo tanto, hace uso de los datos retornados por la función `get_users_functions_products` y recorre los campos para limpiar valores nulos o duplicados.

In [67]:
def get_unique_user_functions_products(user_entitlements):
    unique_access_id = get_users_functions_products(user_entitlements)
    
    return [
        {
            'access_id': access_id,
            'function_id': sorted(list(set(i['function_id'] for i in values if i['function_id'] != ''))),
            'product_id': sorted(list(set(i['product_id'] for i in values if i['product_id'] != '')))
        }
        for access_id, values in unique_access_id.items()
    ]

La función `define_user_profile` se encarga de asignar el perfil a cada usuario en el archivo json normalizado. Lo primero es cargar el archivo de `user_entitlements` para obtener la información respecto a productos y funciones del usuario, luego, se extraen los valores únicos de ambos campos, para ser barridos y determinar que si el usuario tiene un producto llamado `administration` entonces debe ser un administrado, en caso contrario se comparan las acciones y a partir de las mismas se determina el perfil, tal cómo se muestra en la imágen de arriba.

In [68]:
def define_user_profile(user_entitlements_path, company_users_path):
    user_entitlements = open_json_file(user_entitlements_path)
    company_users = open_json_file(company_users_path)
    
    capture_actions = ['cancel', 'input', 'amend']
    verification_actions = ['approve']
    authorization_actions = ['release']
    
    user_functions_products = get_unique_user_functions_products(user_entitlements)
    
    for i in user_functions_products:
        function_id = i['function_id']
        
        company_user_index = get_company_user_index(company_users, i['access_id'])
        if company_user_index is None:
            print(f'access_id {i["access_id"]} not found in company_users')
            continue
        
        if any(product_id == 'administration' for product_id in i['product_id']):
            company_users[company_user_index]['profile_id'] = 'administrator'
        
        elif set(capture_actions).issubset(set(function_id)):
            company_users[company_user_index]['profile_id'] = 'capture'
        elif set(verification_actions).issubset(set(function_id)):
            company_users[company_user_index]['profile_id'] = 'verification'
        elif set(authorization_actions).issubset(set(function_id)):
            company_users[company_user_index]['profile_id'] = 'authorization'
        else:
            company_users[company_user_index]['profile_id'] = 'viewer'
        
    safe_json_file(path=company_users_path, data=company_users)

In [69]:
start = time.perf_counter()

define_user_profile(
    user_entitlements_path = r"normalize-json/users-entitlements.json",
    company_users_path = r"normalize-json/company-users.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.1727 seconds


## Asignación de Profiles - Functions

La función `get_unique_user_functions` se encarga de obtener los valores únicos por usuario para el campo de `function_id`, por lo tanto, hace uso de los datos retornados por la función `get_users_functions_products` y recorre los campos para limpiar valores nulos o duplicados.

In [70]:
def get_unique_user_functions(user_entitlements):
    unique_access_id = get_users_functions_products(user_entitlements)
    
    return [
        {
            'access_id': access_id,
            'function_id': sorted(list(set(i['function_id'] for i in values if i['function_id'] != ''))),
        }
        for access_id, values in unique_access_id.items()
    ]

La función `define_profiles_functions` se encarga de recibir las funciones que tiene cada usuario que se encuentran en `user_entitlements`, luego busca el perfil del mismo dentro de `company_users` y asocia el perfil con las funciones. Por último, realiza una limpieza de combinaciones duplicadas entre perfiles y funciones.

In [71]:
def define_profiles_functions(user_entitlements_path, company_users_path, output_data_path):
    user_entitlements = open_json_file(user_entitlements_path)
    company_users = open_json_file(company_users_path)
        
    unique_user_functions = get_unique_user_functions(user_entitlements)
    
    profiles_ids = {
        "administrator": [],
        "authorization": [],
        "capture": [],
        "verification": [],
        "viewer": []
    }
    
    for i in unique_user_functions:
        company_user_index = get_company_user_index(company_users, i['access_id'])
        profiles_ids[company_users[company_user_index]['profile_id']].append(i)
    
    profiles_functions_json = [
        {
            "id": normalize_str(profile_id + '_' + function_id),
            "profile_id": profile_id,
            "function_id": function_id
        }
        for profile_id, company_users in profiles_ids.items()
        for user in company_users
        for function_id in user['function_id']
    ]
    
    profiles_functions_json = sorted(
        list(map(dict, set(tuple(sorted(d.items(), reverse=True)) for d in profiles_functions_json))),
        key=lambda d: d["profile_id"]
    )
    
    safe_json_file(path=output_data_path, data=profiles_functions_json)

In [72]:
start = time.perf_counter()

define_profiles_functions(    
    user_entitlements_path = r"normalize-json/users-entitlements.json",
    company_users_path = r"normalize-json/company-users.json",
    output_data_path = r"normalize-json/profiles-functions.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.1356 seconds


## Normalize Profiles

La función `create_profiles_json` carga un arreglo de json dentro de un archivo definido

In [73]:
def create_profiles_json(output_data_path):
    profiles = [
        {
            "id": "capture",
            "profile_name": "Capture"
        },
        {
            "id": "verification",
            "profile_name": "Verification"
        },
        {
            "id": "authorization",
            "profile_name": "Authorization"
        },
        {
            "id": "administrator",
            "profile_name": "Administrador"
        },
        {
            "id": "viewer",
            "profile_name": "Viewer"
        }
    ]
    
    safe_json_file(path=output_data_path, data=profiles)

In [74]:
start = time.perf_counter()

create_profiles_json(    
    output_data_path = r"normalize-json/profiles.json"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0011 seconds


## Conversión de JSON a SQL Queries

La función `convert_json_to_sql` se encarga de convertir los registros JSON en sentencias INSERT de SQL, para esto recorre cada objeto y determina que el nombre de las columnas son el nombre de las llaves del diccionario, mientras que los valores deben mantener su formato adaptado a SQL mediante la función `format_value_to_sql`.

In [75]:
def convert_json_to_sql(table_name, json_data_path, sql_queries_path):
    data = open_json_file(json_data_path)

    sql_queries = [
        "INSERT INTO {table_name} ({columns}) VALUES ({values});".format(
            table_name = table_name,
            columns = ', '.join(i.keys()),
            values = ', '.join([format_value_to_sql(v) for v in i.values()])
        )
        for i in data
    ]

    sql_file = io.open(sql_queries_path, 'w')

    for query in sql_queries:
        sql_file.write(query + '\n')

    sql_file.close()

### Accounts

In [76]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'accounts',
    json_data_path = r"normalize-json/accounts.json",
    sql_queries_path = r"sql/accounts.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0022 seconds


### Functions

In [77]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'functions',
    json_data_path = r"normalize-json/functions.json",
    sql_queries_path = r"sql/functions.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0025 seconds


### Products

In [78]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'products',
    json_data_path = r"normalize-json/products.json",
    sql_queries_path = r"sql/products.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0022 seconds


### ProductsAccounts

In [79]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'products_accounts',
    json_data_path = r"normalize-json/products-accounts.json",
    sql_queries_path = r"sql/products-accounts.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0093 seconds


### Company Users

In [80]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'company_users',
    json_data_path = r"normalize-json/company-users.json",
    sql_queries_path = r"sql/company-users.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0022 seconds


### User Entitlements

In [81]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'user_entitlements',
    json_data_path = r"normalize-json/users-entitlements.json",
    sql_queries_path = r"sql/users-entitlements.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0124 seconds


### Profiles

In [82]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'profiles',
    json_data_path = r"normalize-json/profiles.json",
    sql_queries_path = r"sql/profiles.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0022 seconds


### Profiles Functions

In [83]:
start = time.perf_counter()

convert_json_to_sql(
    table_name = 'profiles_functions',
    json_data_path = r"normalize-json/profiles-functions.json",
    sql_queries_path = r"sql/profiles-functions.sql"
)

finish = time.perf_counter()

print(f"Conversion in {finish -start:0.4f} seconds")

Conversion in 0.0030 seconds
